# Introduction to basic functionalities of HOOMD-Organics

## Overview:

This tutorial introduces some basic functionalities of the HOOMD-Organics package including:

- Initializing molecule structures
- Assembling molecules in a box
- Applying force fields
- Running MD simulations using different methods




## Step-by-step guide for running MD simulation for a box of  Polyphenylene sulfide (PPS) polymers:
In this tutorial, we will run a molecular dynamics simulation of Polyphenylene sulfide (PPS) polymers using the HOOMD-Organics package.
HOOMD-Organics uses [`HOOMD-blue`](https://hoomd-blue.readthedocs.io/en/v4.1.0/) simulation engine in the background to run different methods of simulation and utilizes many functionalities from [`mBuild`](https://mbuild.mosdef.org/en/stable/) and [GMSO](https://gmso.mosdef.org/en/stable/) packages to initialize the structures, apply force-fields and prepare necessary information to run a simulation.

In summary, the `HOOMD-Organics` package has three main classes:

-  `Molecule`: This class is used to define the structure of a molecule (for example the structure of a polymer built from a monomer). This class allows us to implement recipies for building complex structures.

- `System`: This class is used to assemble molecules into a box and to capture the initial `gsd` snapshot of the system. It also applies the forcefiled to the system and prepares the required forces for the simulation.

- `Simulation`: This class is used to run the simulation using the `HOOMD-blue` simulation engine. In order to initialize a simulation, a `gsd` snapshot of the system and a list of `Hoomd` forces are required.

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [6]:
# Temporary cell to add hoomd-organics to path. Should be removed once the package is installed via conda.
import sys
sys.path.append("..")

In [5]:
from hoomd_organics.library import PPS, OPLS_AA_PPS
from hoomd_organics import Pack, Simulation

### Step 1: Initializing the Molecule
In this example, we are using the pre-defined recipe for building PPS molecules defined in `HOOMD_Organics` library. The `PPS` class is a subclass of the `Molecule` class. This class includes all the necessary information for building the PPS chains, including the monomer structure and how the monomers bond to create a chain. All we need to specify is the polymer chain length and how many polymer chains we want to create. In this example, we will create a system of 3 PPS chains with a length of 10 monomers.


In [19]:
molecule = PPS(num_mols=3, lengths=10)

### Step 2: Initializing the System
In this step, we will use the `Pack` class, which is a subclass of the `System` class to pack a box of PPS molecules given a density. The system class creates the box, organizes molecules within the box and applies the force-field (if provided) to the system and creates
the initial state of the system in form a `gsd` snapshot. If force-field is provided, this class also gets the list of forces that defines the bonded and non-bonded interactions between the particles.

In this example, we pass the molecule object created in step 1 to pack a box with density=0.8. `Pack` class calls mBuild's `fill_box` method which puts the molecules into a box in a random manner without overlapping. We use the pre-defined `OPLS` force-field class which includes all the parameters found in the OPLS xml force-field file to parameterize particle interactions. We also specify the `r_cut` parameter which is the cutoff distance for the non-bonded interactions. If `auto_scale` is set to `True`, all the parameters defined in forces (for example, the `epsilon` parameter in Leonard-Jones potential) will be scaled.

In [20]:
system = Pack(molecules=molecule, force_field=OPLS_AA_PPS(), density=0.8, r_cut=2.5, auto_scale=True)

We can obtain the `gsd` snapshot of the system by calling the `system.snapshot` attribute.


In [21]:
system.hoomd_snapshot

We can also obtain the list of HOOMD forces applied to the system by calling the `system.forces` attribute.

In [22]:
system.hoomd_forcefield

### Step 3: Running the Simulation

Using the snapshot and forces provided by the system class, we can initialize the simulation. The `Simulation` class  logs snapshots of the simulation in form of a `gsd` trajectory file while running simulation. The `gsd_write_freq` specifies the frequency of saving snapshots into the gsd file. This class also logs other simulation data such as timestep, potential energy, kinetic temperature, pressure and volume into a text file. The frequency for logging these information can be set by `log_write_freq` parameter.

In [23]:
sim = Simulation(initial_state=system.hoomd_snapshot, forcefield=system.hoomd_forcefield, gsd_write_freq=100, log_write_freq=100)

Initializing simulation state from a snapshot.


We can now run the simulation for 1000 time steps using the NVT ensemble at a given scaled temperature of 1.0.

In [25]:
sim.run_NVT(n_steps=1000, kT=1.0, tau_kt=0.01)

Step 0 of 1000; TPS: 0.0; ETA: inf minutes
Step 100 of 1000; TPS: 1100.57; ETA: 0.0 minutes
Step 200 of 1000; TPS: 1164.25; ETA: 0.0 minutes
Step 300 of 1000; TPS: 1189.75; ETA: 0.0 minutes
Step 400 of 1000; TPS: 1199.7; ETA: 0.0 minutes
Step 500 of 1000; TPS: 1204.66; ETA: 0.0 minutes
Step 600 of 1000; TPS: 1208.0; ETA: 0.0 minutes
Step 700 of 1000; TPS: 1212.62; ETA: 0.0 minutes
Step 800 of 1000; TPS: 1218.05; ETA: 0.0 minutes
Step 900 of 1000; TPS: 1218.94; ETA: 0.0 minutes


The simulation class also allows user to run the simulation under different conditions such as NPT ensemble, NVE ensemble, Langevin dynamics. Checkout `hoomd_organics/base/simulation.py` for more functionalities.


In the upcoming tutorial, we will explore a selection of features offered by the `HOOMD-Organics` package, highlighting how they can be customized to meet specific research requirements.